<a href="https://colab.research.google.com/github/sumanyurosha/tensorflow-specialization/blob/master/Course3/Week2/Lesson1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds

In [4]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [7]:
import numpy as np

training_labels = []
training_sentences = []

testing_labels = []
testing_sentences = []

for s, l in imdb["train"]:
    training_sentences.append(s.numpy().decode("utf8"))
    training_labels.append(l)

for s, l in imdb["test"]:
    testing_sentences.append(s.numpy().decode("utf8"))
    testing_labels.append(l)

training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

print(len(training_sentences))
print(len(testing_sentences))

25000
25000


In [13]:
vocab_size = 10000
oov_token = "<OOV>"
embedding_dim = 16
max_len = 120
trunc_type = "post"

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
training_word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(test_sequences, truncating=trunc_type, maxlen=max_len)

In [19]:
print(sequences[1])
print(padded[1])

[11, 26, 75, 571, 6, 805, 2354, 313, 106, 19, 12, 7, 629, 686, 6, 4, 2219, 5, 181, 584, 64, 1454, 110, 2263, 3, 3951, 21, 2, 1, 3, 258, 41, 4677, 4, 174, 188, 21, 12, 4078, 11, 1578, 2354, 86, 2, 20, 14, 1907, 2, 112, 940, 14, 1811, 1340, 548, 3, 355, 181, 466, 6, 591, 19, 17, 55, 1817, 5, 49, 14, 4044, 96, 40, 136, 11, 972, 11, 201, 26, 1046, 171, 5, 2, 20, 19, 11, 294, 2, 2155, 5, 10, 3, 283, 41, 466, 6, 591, 5, 92, 203, 1, 207, 99, 145, 4382, 16, 230, 332, 11, 2486, 384, 12, 20, 31, 30]
[   0    0    0    0    0    0    0    0   11   26   75  571    6  805
 2354  313  106   19   12    7  629  686    6    4 2219    5  181  584
   64 1454  110 2263    3 3951   21    2    1    3  258   41 4677    4
  174  188   21   12 4078   11 1578 2354   86    2   20   14 1907    2
  112  940   14 1811 1340  548    3  355  181  466    6  591   19   17
   55 1817    5   49   14 4044   96   40  136   11  972   11  201   26
 1046  171    5    2   20   19   11  294    2 2155    5   10    3  283
   41  4

In [23]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(6, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss="binary_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

model.fit(padded, training_labels, epochs=15, validation_data=(test_padded, testing_labels))

Epoch 1/15
782/782 [==============================] - 4s 5ms/step - loss: 0.5829 - accuracy: 0.7210 - val_loss: 0.4482 - val_accuracy: 0.8108
Epoch 2/15
782/782 [==============================] - 4s 5ms/step - loss: 0.3711 - accuracy: 0.8465 - val_loss: 0.3793 - val_accuracy: 0.8304
Epoch 3/15
782/782 [==============================] - 4s 5ms/step - loss: 0.3070 - accuracy: 0.8734 - val_loss: 0.3668 - val_accuracy: 0.8370
Epoch 4/15
782/782 [==============================] - 4s 5ms/step - loss: 0.2722 - accuracy: 0.8909 - val_loss: 0.3687 - val_accuracy: 0.8387
Epoch 5/15
782/782 [==============================] - 4s 5ms/step - loss: 0.2482 - accuracy: 0.9026 - val_loss: 0.3768 - val_accuracy: 0.8367
Epoch 6/15
782/782 [==============================] - 4s 5ms/step - loss: 0.2290 - accuracy: 0.9112 - val_loss: 0.3886 - val_accuracy: 0.8344
Epoch 7/15
782/782 [==============================] - 4s 5ms/step - loss: 0.2130 - accuracy: 0.9193 - val_loss: 0.4037 - val_accuracy: 0.8327
Epoch 